In [ ]:
import sys
from collections import OrderedDict
from pyexcel_ods import get_data
from config_rm import *
from redminelib import Redmine

In [ ]:
redmine = Redmine(rm_server_url,key=rm_key_txt)
projects = redmine.project.all()

In [ ]:
print("Proyectos:")
for p in projects:
    print ("\t",p.identifier," \t| ",p.name)

my_project = redmine.project.get(rm_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)

In [ ]:
import datetime
def date_by_adding_business_days(from_date, add_days):
    business_days_to_add = add_days
    current_date = from_date
    while business_days_to_add > 0:
        current_date += datetime.timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5: # sunday = 6
            continue
        business_days_to_add -= 1
    return current_date


my_project = redmine.project.get(rm_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)
done_issues = []
issues_list = []
issues_number = len(my_project.issues)
print("issues_number",issues_number)
childissues = []
for tsk in my_project.issues:
    if (hasattr(tsk, 'parent')):
        thisparent = tsk.parent
    else:
        thisparent = None
    
    if thisparent is None:
        thisTskRelations = tsk.relations
        for rel in thisTskRelations:
            print(rel.relation_type, tsk.id, rel.issue_to_id, rel.issue_id)

        thisTskInputRelations = list(filter(lambda x: x.issue_to_id == tsk.id and x.relation_type=='precedes', thisTskRelations))
        if len(thisTskInputRelations)>0:
            print("bypass "+tsk.subject)
            issues_list += [{'tsk': tsk, 'relations': thisTskInputRelations}]

        else:
            print("process non-dependent "+tsk.subject)
            done_issues += [tsk.id]
            print("---> done:",tsk)            
            if not(hasattr(tsk, 'start_date')):
                tsk.start_date = datetime.date.today()
                
            if len(tsk.children)>0:
                print("Children:",tsk.children)
                for c in tsk.children:
                    print("child:",c)
                    c2 = redmine.issue.get(c.id)                    
                    done_issues += [c.id]
                    print("---> done child:",c2)
                    c2.start_date = tsk.start_date
                    if not(hasattr(c2, 'estimated_hours')):
                        hours = 0
                    else:
                        hours = c2.estimated_hours

                    print("calculate due date from "+str(c2.start_date)+" + "+str(hours))
                    tmp = date_by_adding_business_days(c2.start_date, hours/8)
                    print("result = "+str(tmp))
                    c2.due_date = tmp
                    c2.save()

            else:
                if not(hasattr(tsk, 'estimated_hours')):
                    hours = 0
                else:
                    hours = tsk.estimated_hours
                print("calculate due date from "+str(tsk.start_date)+" + "+str(hours))
                tmp = date_by_adding_business_days(tsk.start_date, hours/8)
                print("result = "+str(tmp))
                tsk.due_date = tmp

            tsk.save()

    
print (done_issues)
print (issues_list)


In [ ]:
print(len(done_issues),issues_number)
while len(done_issues)<issues_number:
    for tsk1 in my_project.issues:
        print("task:",tsk1)
        if (tsk1.id not in done_issues):
            if len(tsk1.children) <= 0:
                tsk = redmine.issue.get(tsk1.id)
                print(tsk)
                if not(hasattr(tsk, 'start_date')):
                    tsk.start_date = datetime.date.today()

                if not(hasattr(tsk, 'estimated_hours')):
                    hours = 0
                else:
                    hours = tsk.estimated_hours

                print("calculate due date from "+str(tsk.start_date)+" + "+str(hours))
                tmp = date_by_adding_business_days(tsk.start_date, hours/8)
                print("result = "+str(tmp))
                tsk.due_date = tmp        
                tsk.save()
                done_issues += [tsk]
            else:
                done_issues += [tsk]
                print("doneissues",len(done_issues))



        
    

In [ ]:
print("Done")